In [7]:
#EDA
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#to download json_lines: python -m pip install json-lines
#pip install textstat
#pip install --upgrade gensim
from pathlib import Path
import json_lines, json, copy
import numpy as np
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim.downloader as api
import gensim
from nltk.util import ngrams 
import nltk
from nltk.corpus import stopwords
from nltk import download
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from gensim import corpora

import csv
from textstat.textstat import textstat
stop_words = stopwords.words('english')
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.tokenize import TweetTokenizer


#TODO combine lists into csv

#toy example
#rows = zip(list1,list2,list3,list4,list5)
#with open(newfilePath, "w") as f:
#    writer = csv.writer(f)
#    for row in rows:
#        writer.writerow(row)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/EthanHsu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
word_vectors = api.load("glove-wiki-gigaword-100")

In [321]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)


In [268]:
nlp_en = spacy.load('en_core_web_md')

In [269]:
nlp_lg = spacy.load('en_vectors_web_lg')

In [270]:
nlp_core_lg = nlp_en = spacy.load('en_core_web_lg')

In [8]:
#assorted cleaning
stopWords = stopwords.words('english')

def stem_tokens(tokens, ps):
    stemmed = []
    for item in tokens:
        stemmed.append(ps.stem(item))
    return stemmed
ps = PorterStemmer()

def lemmatize_tokens(tokens, wnl):
    lemmatized = []
    for item in tokens:
        lemmatized.append(wnl.lemmatize(item))
    return lemmatized
wnl = WordNetLemmatizer()


In [9]:
raw_instance_filePath = Path("../clickbait_dataset/instances_train.jsonl")
raw_instance_dir = Path("../clickbait_dataset")
raw_truth_filePath = Path("../clickbait_dataset/truth_train.jsonl")
raw_truth_dir = Path("../clickbait_dataset")

In [272]:
#open with json - raw_instance_train
raw_instance_train=[]
with open('../clickbait_dataset/instances_train.jsonl','rb') as raw_instance_file: #opening file with binary(rb) mode.
    for x in raw_instance_file:
        raw_instance_train.append(json.loads(x))
raw_instance_train[0]

### Fields in instances.jsonl:
##{
#"id" : "<instance id>",
#"postTimestamp" : "<weekday> <month> <day> <hour>:<minute>:<second><time_offset> <year>",
#"postText" : ["<text of the tweet post with links removed>"],
#"postMedia" : ["<path to a file in the media archive>"],
#"targetTitle" : "<title of target news article>",
#"targetDescription": "<description tag of target news article>",
#"targetKeywords" : "<keywords tag of target news article>",
#"targetParagraphs" : ["<text of the i-th paragraph in the target article>"],
#"targetCaptions" : ["<caption of the i-th image in the target article>"]}
### Fields in truth.jsonl:
#{
#"id" : "<instance id>",
#"truthJudgments" : [<number in [0,1]>],
#"truthMean" : <number in [0,1]>,
#"truthMedian" : <number in [0,1]>,
#"truthMode" : <number in [0,1]>,
#truthClass" : "clickbait | no-clickbait"
#}

{'id': '0',
 'postMedia': [],
 'postText': ["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"],
 'postTimestamp': 'Tue Jun 09 16:31:10 +0000 2015',
 'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The Ap

In [273]:
_instance_train = []
print(len(raw_instance_train))
for id_, instance in enumerate(raw_instance_train):
    if instance['targetParagraphs'] != []:
        _instance_train.append(instance)
raw_instance_train = _instance_train

17581


In [11]:
#open with json - raw_truth_train

raw_truth_train=[]
with open('../clickbait_dataset/truth_train.jsonl','rb') as raw_truth_file: #opening file with binary(rb) mode.
    for x in raw_truth_file:
        raw_truth_train.append(json.loads(x))
raw_truth_train[7]


{'id': '7',
 'truthClass': 'no-clickbait',
 'truthJudgments': [0.33333334, 0.0, 0.33333334, 0.33333334, 0.0],
 'truthMean': 0.2,
 'truthMedian': 0.33333334,
 'truthMode': 0.33333334}

In [12]:
#check the properties
print(type(raw_instance_train))    #type for whole data: list
print(type(raw_instance_train[0]))  #type for each element of the liast: dict
print(len(raw_instance_train))
print('Check Dict. Indexes for instance_train: ' + str(raw_instance_train[0].keys()))
print('Check Dict. Indexes for truth_train: ' + str(raw_truth_train[0].keys()))

<class 'list'>
<class 'dict'>
17581
Check Dict. Indexes for instance_train: dict_keys(['targetCaptions', 'targetDescription', 'postText', 'postTimestamp', 'postMedia', 'targetParagraphs', 'targetTitle', 'id', 'targetKeywords'])
Check Dict. Indexes for truth_train: dict_keys(['truthMedian', 'truthMean', 'truthMode', 'truthClass', 'truthJudgments', 'id'])


In [13]:
raw_instance_train = [json.loads(line) for line in open("../clickbait_dataset/instances_train.jsonl")]
raw_truth_train = [json.loads(line) for line in open("../clickbait_dataset/truth_train.jsonl")]

In [287]:
#separate into two list (clickbait/ non_clickbait)
clickbait_train = []
non_clickbait_train = []
for x in range(0, len(raw_instance_train)):
    if raw_truth_train[x]['truthClass'] == 'clickbait':
        temp_dict = copy.deepcopy(raw_instance_train[x])
        temp_dict.update(raw_truth_train[x])
        clickbait_train.append(temp_dict)
    else:
        raw_instance_train[x].update(raw_truth_train[x])
        non_clickbait_train.append(raw_instance_train[x])
        

In [288]:
#clickbait_train
clickbait_train[0]

{'id': '2',
 'postMedia': [],
 'postText': ['U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.'],
 'postTimestamp': 'Fri Jun 12 23:36:05 +0000 2015',
 'targetCaptions': ['US to vote for Ali in FIFA election and not Blatter',
  'US to vote for Ali in FIFA election and not Blatter',
  "FILE - This Oct. 10, 2014, file photo shows Sunil Gulati, president of the United States Soccer Federation, during a press conference in Bristol, Conn. The United States says it will vote for Jordan's Prince Ali bin Al-Hussein for FIFA president Friday, May 29, 2015 and not for incumbent Sepp Blatter. (AP Photo/Elise Amendola, File)"],
 'targetDescription': "A U.S. Senator's scathing letter questioned U.S. Soccer's inadequate handling of Solo's domestic violence charges. It's time for Sunil Gulati to respond.",
 'targetKeywords': '',
 'targetParagraphs': ["WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns 

In [290]:
#non_clickbait_train
non_clickbait_train[10]

{'id': '13',
 'postMedia': ['media/609998126737395712.jpg'],
 'postText': ['Britain forced to withdraw spies after Russia and China access files stolen by Snowden'],
 'postTimestamp': 'Sun Jun 14 08:17:41 +0000 2015',
 'targetCaptions': ['GCHQ-AFPGetty.jpg',
  'GettyImages-74018446.jpg',
  '4-Edward-Snowden-AP.jpg',
  'GettyImages-2428443.jpg'],
 'targetDescription': 'The news that Russia and China have gained access to secret information in files leaked by whistleblower Edward Snowden has been called a "huge catastrophe" by a former CIA field officer.',
 'targetKeywords': ', Home News, UK, News',
 'targetParagraphs': ["UK security chiefs have warned that the former NSA contractor's actions have compromised anti-terror activity",
  'The news that Russia and China have gained access to secret information in files leaked by whistleblower Edward Snowden has been called a "huge catastrophe" by a former CIA field officer.',
  'Speaking on BBC Radio 5 Live this morning, Robert Baer, an autho

In [17]:
#Post Text tokenizing and word count manually
postTextTokens = []
postTextTokensNoPun = []
postTextCountTokens = []

for x in range(0, len(raw_instance_train)):
    tokens = word_tokenize(raw_instance_train[x]['postText'][0])
    postTextCountTokens.append([x for x in tokens if x != []])
    postTextTokensNoPun.append([c for c in tokens if c not in punctuation])
#print(postTextTokens[0])
#print(len(postTextTokens[0]))

In [18]:
def doc_parser(document):
    #print(document)
    #print(porter_stemmer.stem(document))
    #print(stem_doc)
    tokens = word_tokenize(document)
    non_empty_tokens = [x for x in tokens if x != []]
    clean_tokens = [c for c in non_empty_tokens if c not in punctuation and stop_words]
    #non_apos_tokens = [c.replace("'", "") for c in non_empty_tokens]
    return clean_tokens

In [19]:
def doc_punc_parser(document):
    #print(document)
    #print(porter_stemmer.stem(document))
    #print(stem_doc)
    tokens = word_tokenize(document)
    non_empty_tokens = [x for x in tokens if x != []]
    #clean_tokens = [c for c in non_empty_tokens if c not in punctuation and stop_words]
    #non_apos_tokens = [c.replace("'", "") for c in non_empty_tokens]
    return non_empty_tokens

In [416]:
# remove common words and tokenize
def documentParser(document):  
    #print(punctuation)
    stoplist = set('for a of the and to in'.split())
    tokens = [word for word in document.lower().split() if word not in stoplist and punctuation]
    #print(tokens)
    #stem_tokens = [porter_stemmer.stem(a) for a in tokens]
    #print("stem",stem_tokens)
    non_apos_tokens = [c.replace("'", "").replace(":", "").replace(".", "").replace(",", "") for c in tokens]
    return non_apos_tokens

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
def sklearn_TfidfVectorizer(document):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(document)
    #print(X.shape)
    return vectorizer.get_feature_names()
    

In [22]:
def tweet_Tokenizer(document):
    tknzr = TweetTokenizer()
    #tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
    
    return [c for c in tknzr.tokenize(document) if c not in punctuation]

In [23]:
print(raw_instance_train[0]['postText'])
print("---------------------")
print("postTextCountTokens: ", postTextCountTokens[0], "\n")
print("postTextTokensNoPun: ", postTextTokensNoPun[0], "\n")
print("doc_parser(): ",doc_parser(raw_instance_train[0]['postText'][0]), "\n")
print("doc_punc_parser(): ", doc_punc_parser(raw_instance_train[0]['postText'][0]), "\n")
print("documentParser(): ", documentParser(raw_instance_train[0]['postText'][0]), "\n")
print("sklearn_TfidfVectorizer(): ",sklearn_TfidfVectorizer(raw_instance_train[0]['postText']), "\n")
print("Tweet_Tokenizer(): ", tweet_Tokenizer(raw_instance_train[0]['postText'][0]), "\n")

["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"]
---------------------
postTextCountTokens:  ['Apple', "'s", 'iOS', '9', "'App", 'thinning', "'", 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost'] 

postTextTokensNoPun:  ['Apple', "'s", 'iOS', '9', "'App", 'thinning', 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost'] 

doc_parser():  ['Apple', "'s", 'iOS', '9', "'App", 'thinning', 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost'] 

doc_punc_parser():  ['Apple', "'s", 'iOS', '9', "'App", 'thinning', "'", 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost'] 

documentParser():  ['apples', 'ios', '9', 'app', 'thinning', 'feature', 'will', 'give', 'your', 'phones', 'storage', 'boost'] 

sklearn_TfidfVectorizer():  ['app', 'apple', 'boost', 'feature', 'give', 'ios', 'phone', 'storage', 'thinning', 'will', 'your'] 

Tweet_Tokenizer():  ["Apple's", 'iOS', '9', 'App', 

In [24]:
print(raw_instance_train[0]['targetTitle'])
documentParser(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


['apple',
 'gives',
 'back',
 'gigabytes',
 'ios',
 '9',
 'app',
 'thinning',
 'feature',
 'will',
 'finally',
 'give',
 'your',
 'phones',
 'storage',
 'boost']

In [25]:
countLongestWord = []

def count_longest_word(word_list):  
    longest_word =  max(word_list , key=len)
    return len(longest_word)

for x in range(0, 1751):
    countLongestWord.append(count_longest_word(postTextTokensNoPun[x]))

print(max(countLongestWord))


26


In [26]:
print(postTextTokensNoPun[915])
count_longest_word(postTextTokensNoPun[915])

['Fury', 'as', 'men', 'throw', 'a', 'cat', 'into', 'a', 'crocodile-infested', 'pond']


18

In [27]:
#Lexicon (Word) Count
targetTitleCountWords = []
for x in range(0, len(raw_instance_train)):
    targetTitleCountWords.append(textstat.lexicon_count(raw_instance_train[x]['targetTitle'][0], removepunct=True))

In [28]:
#Grade Level reading difficulty
#Coleman-Liau index, Gunning FOG Formula, Linsear Write Formula
targetTitle_coleman_liau_index = []
targetTitle_gunning_fog = []
targetTitle_linsear_write_formula = []
for x in range(0, len(raw_instance_train)):
    targetTitle_coleman_liau_index.append(textstat.coleman_liau_index(raw_instance_train[x]['targetTitle']))
    targetTitle_gunning_fog.append(textstat.gunning_fog(raw_instance_train[x]['targetTitle']))
    targetTitle_linsear_write_formula.append(textstat.linsear_write_formula(raw_instance_train[x]['targetTitle']))
targetTitle_coleman_liau_index = []


In [29]:
print(raw_instance_train[0]['targetTitle'])
textstat.coleman_liau_index(raw_instance_train[0]['targetTitle'])


Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


12.12

In [30]:
print(raw_instance_train[0]['targetTitle'])
textstat.gunning_fog(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


20.564705882352943

In [31]:
print(raw_instance_train[0]['targetTitle'])
textstat.linsear_write_formula(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


6.5

In [56]:
def avg_sim(title, body):
    title = api(title)
    return np.mean(map(lambda x: title.similarity(api(x)), body))

In [65]:
#calculating similarity between titles and body.
import spacy
#nlp = spacy.load('../GoogleNews-vectors-negative300.bin')

def avg_doc_sim(nlp, title, body):
#    similarity = word_vectors.n_similarity(title, body)
#    return "{:.4f}".format(similarity)
    print(title)
    title = nlp(title)
    print(title)
    
    return np.mean(map(lambda x: title.similarity(nlp(x)), body))

In [ ]:
doc1 = nlp_en(u"'Obama speaks to the media in Illinois'")
doc2 = nlp_en(u"The president greets the press in Chicago")
doc1.similarity(doc2)

In [249]:
#use this one
def spacy_acg_sim(title, body):
    body_list = []
    title = nlp_en(title)
    for b in body:
        body_list.append(title.similarity(nlp_en(b)))        
    return np.mean(body_list) 

In [227]:
np.mean([0, 0.1, 0.8, 0.9])

0.45000000000000001

In [228]:
sentence_obama = 'Obama speaks to the media in Illinois'#.lower().split()
sentence_president = 'The president greets the press in Chicago'#.lower().split()

In [229]:
title_doc1 = nlp_en(raw_instance_train[0]['targetTitle'])
paragraph_doc2 = nlp_en(raw_instance_train[0]['targetParagraphs'][0])
title_doc1.similarity(paragraph_doc2)

0.85945738934493399

In [230]:
#avg_similairty_result
spacy_acg_sim(clickbait_train[10]['targetTitle'],clickbait_train[10]['targetParagraphs'])

0.6058277647979724

In [235]:
train_list = []
train_list_n = []
print(len(raw_instance_train))
for x in range(0, 100):
    #TODO BLOCK NULL
    train_list.append(spacy_acg_sim(clickbait_train[x]['targetTitle'],clickbait_train[x]['targetParagraphs']))
for x in range(0, 100):
    #TODO BLOCK NULL
    train_list_n.append(spacy_acg_sim(non_clickbait_train[x]['targetTitle'],non_clickbait_train[x]['targetParagraphs']))
np.mean(train_list)
np.mean(train_list_n)

17490


0.76707725013332617

In [236]:
print(np.mean(train_list))
print(np.mean(train_list_n))

0.757950531111
0.767077250133


In [68]:
#checking glove-wiki-gigaword-100 and ../GoogleNews-vectors-negative300.bin performance
#print("glove-wiki-gigaword-100:",avg_doc_sim(word_vectors,sentence_obama,sentence_president))
print("GoogleNews:", avg_doc_sim(model,raw_instance_train[0]['targetTitle'], raw_instance_train[0]['targetParagraphs']))
#raw_instance_train

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


TypeError: 'Word2VecKeyedVectors' object is not callable

In [588]:
print("glove-wiki-gigaword-100:", avg_doc_sim(word_vectors,['sushi', 'shop'], ['japanese', 'restaurant']))
print("GoogleNews:", avg_doc_sim(model,['sushi', 'shop'], ['japanese', 'restaurant']))

glove-wiki-gigaword-100: 0.7067
GoogleNews: 0.5984


In [804]:
import re

def cleanhtml_(raw_html):
    cleanr = re.compile('<.*?>')
    print(raw_html.replace(u'\xa0',u''))
    return raw_html.replace(u'\xa0',u'')
    cleantext_ = re.sub(cleanr, '', clean_html)
    #return (cleantext_)
    return cleantext_

In [805]:
str1 = "lives.\xa0"

In [806]:
cleanhtml_(str1)

lives.


'lives.'

In [815]:
#Parsing targetTitle and targetParagraphs
#print([raw_instance_train[0]['targetTitle']])
c_target_title_ = tweet_Tokenizer(raw_instance_train[0]['targetTitle'])
c_target_paragraph_ = []
for n in raw_instance_train[0]['targetParagraphs']:
    n = n.replace(u'\xa0',u'').replace(':', ' ')
    if(n != ""):
        #n = [n]
        c_target_paragraph_ += tweet_Tokenizer(n)

In [816]:
#review parsing result for targetTitle and targetParagraphs
print(c_target_title_) #target title
print("------")
print(c_target_paragraph_) # target paragraph

['Apple', 'gives', 'back', 'gigabytes', 'iOS', '9', 'app', 'thinning', 'feature', 'will', 'finally', 'give', 'your', "phone's", 'storage', 'a', 'boost']
------
['Paying', 'for', 'a', '64GB', 'phone', 'only', 'to', 'discover', 'that', 'this', 'is', 'significantly', 'reduced', 'by', 'system', 'files', 'and', 'bloatware', 'is', 'the', 'bane', 'of', 'many', 'smartphone', "owner's", 'lives', 'And', 'the', 'issue', 'became', 'so', 'serious', 'earlier', 'this', 'year', 'that', 'some', 'Apple', 'users', 'even', 'sued', 'the', 'company', 'over', 'it', 'But', 'with', 'the', 'launch', 'of', 'iOS', '9', 'Apple', 'is', 'hoping', 'to', 'address', 'storage', 'concerns', 'by', 'introducing', 'a', 'feature', 'known', 'as', 'app', 'thinning', 'It', 'has', 'been', 'explained', 'on', 'the', 'watchOS', 'Developer', 'Library', 'site', 'and', 'is', 'aimed', 'at', 'developers', 'looking', 'to', 'optimise', 'their', 'apps', 'to', 'work', 'on', 'iOS', 'and', 'the', 'watchOS', 'It', 'ensures', 'apps', 'use', 'th

In [817]:
#measure title and body's similarity
#error "word 'watchos' not in vocabulary"
avg_doc_sim(model,c_target_title_, c_target_paragraph_)

KeyError: "word 'phone's' not in vocabulary"

In [337]:
#targetCaptions
targetCaptions_documents = raw_instance_train[0]['targetCaptions']
targetCaptions_documents

["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
 "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
 "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The App Store hosts these resources on Apple servers and manages the downloads for the developer and user.\xa0This will also increase how quickly an app downloads",
 "The guidelines also discuss so-calle

In [336]:
# find common words and tokenize
def pre_word_count(documents):
    stoplist = set('for a of the and to in'.split())
    texts = [[word for word in document.lower().split() if word not in stoplist]
             for document in documents]
    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [[token for token in text if frequency[token] > 1]
             for text in texts]
    return texts

In [210]:
#count each word in a document
def word_Count(parsed_document):
    dictionary = corpora.Dictionary(parsed_document)
    return dictionary.token2id

In [338]:
#finding common words
targetCaptions_parsed_document = pre_word_count(targetCaptions_documents)
targetCaptions_parsed_document

[["'app",
  "thinning'",
  'will',
  'be',
  'supported',
  'on',
  "apple's",
  'ios',
  '9',
  'later',
  'models.',
  'it',
  'ensures',
  'apps',
  'use',
  'lowest',
  'amount',
  'storage',
  'space',
  'on',
  'device',
  'by',
  'only',
  'downloading',
  'parts',
  'it',
  'needs',
  'run',
  'on',
  'individual',
  'handsets.',
  'it',
  "'slices'",
  'app',
  'into',
  "'app",
  "variants'",
  'that',
  'only',
  'need',
  'access',
  'specific',
  'files',
  'on',
  'that',
  'specific',
  'device'],
 ["'app",
  "thinning'",
  'will',
  'be',
  'supported',
  'on',
  "apple's",
  'ios',
  '9',
  'later',
  'models.',
  'it',
  'ensures',
  'apps',
  'use',
  'lowest',
  'amount',
  'storage',
  'space',
  'on',
  'device',
  'by',
  'only',
  'downloading',
  'parts',
  'it',
  'needs',
  'run',
  'on',
  'individual',
  'handsets.',
  'it',
  "'slices'",
  'app',
  'into',
  "'app",
  "variants'",
  'that',
  'only',
  'need',
  'access',
  'specific',
  'files',
  'on',
 

In [339]:
#word count
word_Count(targetCaptions_parsed_document)

{'%': 83,
 "'app": 0,
 "'concealing,": 115,
 "'on-demand": 37,
 "'purge": 69,
 "'slices'": 1,
 "(apple's": 70,
 '(illustrated)': 84,
 '16gb': 116,
 '20%': 117,
 '54%': 85,
 '56gb': 86,
 '5c': 87,
 '6': 88,
 '64gb': 89,
 '79%': 90,
 '87.5': 91,
 '8gb,': 92,
 '9': 2,
 'access': 3,
 'accusing': 118,
 'advertised': 119,
 'after': 93,
 'allows': 38,
 'also': 39,
 'amount': 4,
 'an': 40,
 'app': 5,
 'apple': 41,
 "apple's": 6,
 'apps': 7,
 'apps,': 94,
 'are': 42,
 'available': 120,
 'available.': 121,
 'be': 8,
 'between': 95,
 'by': 9,
 'capacity': 122,
 'developer': 43,
 'developers': 44,
 'device': 10,
 "disclose'": 123,
 'discuss': 45,
 'disgruntled': 124,
 'disk': 71,
 'downloading': 11,
 'downloads': 46,
 'drop': 96,
 'earlier': 125,
 'ensures': 12,
 'failing': 126,
 'features': 47,
 'filed': 127,
 'files': 13,
 'free': 97,
 'free.': 98,
 'from': 48,
 'galaxy': 99,
 'giant': 128,
 'graph': 129,
 'guidelines': 49,
 'handsets,': 100,
 'handsets.': 14,
 'hosts': 50,
 'how': 51,
 'include

In [ ]:
#pos_tag

In [204]:
text = word_tokenize("And now for something completely different")
nltk.pos_tag(text)

[('And', 'CC'),
 ('now', 'RB'),
 ('for', 'IN'),
 ('something', 'NN'),
 ('completely', 'RB'),
 ('different', 'JJ')]

In [399]:
#clickbait_train, emoji
def pos_1gram(data, title):
    postag_words = word_tokenize(data.get(title)[0])
    return nltk.pos_tag(postag_words)

pos_tag_list = []
#for single_data in clickbait_train:
    #print(pos_2gram(single_data, "postText"))

In [401]:
pos_1gram(clickbait_train[0], "postText")

[('U.S.', 'NNP'),
 ('Soccer', 'NNP'),
 ('should', 'MD'),
 ('start', 'VB'),
 ('answering', 'VBG'),
 ('tough', 'JJ'),
 ('questions', 'NNS'),
 ('about', 'IN'),
 ('Hope', 'NNP'),
 ('Solo', 'NNP'),
 (',', ','),
 ('@', 'NNP'),
 ('eric_adelson', 'NN'),
 ('writes', 'NNS'),
 ('.', '.')]

In [566]:
#calculate tags in a data
def cal_pos_1grams_tag(data, title, tag):
    i = 0
    pos_list = []
    postag_words = word_tokenize(data.get(title)[0])
    pos_list = nltk.pos_tag(postag_words)
    for tags in pos_list:
        print("tags: ", tags)
        if tags[1] == tag:
            i+=1
    return i   

In [565]:
cal_pos_1grams_tag(clickbait_train[0], "postText", "NNP")

tags:  ('U.S.', 'NNP')
tags:  ('Soccer', 'NNP')
tags:  ('should', 'MD')
tags:  ('start', 'VB')
tags:  ('answering', 'VBG')
tags:  ('tough', 'JJ')
tags:  ('questions', 'NNS')
tags:  ('about', 'IN')
tags:  ('Hope', 'NNP')
tags:  ('Solo', 'NNP')
tags:  (',', ',')
tags:  ('@', 'NNP')
tags:  ('eric_adelson', 'NN')
tags:  ('writes', 'NNS')
tags:  ('.', '.')


5

In [561]:
#yutarochan's code
def pos_2gram(title, p1, p2):
    pos_list = pos_tag(word_tokenize(title))
    #print("-1: ", pos_list[:-1])
    #print("1: ", pos_list[1:])
    #print("zip: ", list(zip(pos_list[:-1], pos_list[1:])))
    #print(list(zip(pos_list[:-1], pos_list[1:]))[0][1])
    #print(list(zip(pos_list[:-1], pos_list[1:]))[1][1])
    return sum(map(lambda x: x[0][1] == p1 and x[1][1] == p2, zip(pos_list[:-1], pos_list[1:])))

In [562]:
clickbait_train[0]['targetTitle']

'U.S. Soccer should start answering tough questions about Hope Solo'

In [563]:
pos_2gram(clickbait_train[0]['targetTitle'], 'NNP', 'NNP')

2

In [470]:
#revised pos_2grams
def pos_2gram_sum(title, p1, p2):
    pos_2g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2g_num += 1
    return pos_2g_num

In [471]:
pos_2gram_sum(clickbait_train[0]['targetTitle'], 'NNP', 'NNP')

2

In [472]:
#check pos_2grams
def pos_2gram_check(title, p1, p2):
    pos_2gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2gram_list.append([pos_z[0], pos_z[1]])
    return pos_2gram_list

In [473]:
print(pos_2gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP'))

[[('U.S.', 'NNP'), ('Soccer', 'NNP')], [('Hope', 'NNP'), ('Solo', 'NNP')]]


In [484]:
#revised pos_3grams
def pos_3gram_sum(title, p1, p2, p3):
    pos_3g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3g_num += 1
    return pos_3g_num

In [493]:
print(pos_3gram_sum(clickbait_train[0]['targetTitle'], 'NNP', 'NNP', 'NNP'))

0


In [494]:
#check pos_3grams
def pos_3gram_check(title, p1, p2, p3):
    pos_3gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3gram_list.append([pos_z[0], pos_z[1],pos_z[2]])
    return pos_3gram_list

In [495]:
print(pos_3gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP', 'NNP'))

[]


In [507]:
#just downloading some nltk samples
nltk.download('brown')
nltk.corpus.brown.tagged_words(categories='news')

[nltk_data] Downloading package brown to /Users/EthanHsu/nltk_data...
[nltk_data]   Package brown is already up-to-date!


[('The', 'AT'), ('Fulton', 'NP-TL'), ...]

In [508]:
#just downloading some nltk samples
nltk_list = list(nltk.corpus.brown.tagged_words(categories='news'))

In [552]:
target_para_list = []
for n in clickbait_train[0]['targetParagraphs']:
    target_para_ = doc_punc_parser(n)
    #print(type(target_para_list))
    #print(target_para_)
    target_para_list = target_para_list + target_para_
    #print(n)
    #print(nltk.pos_tag(n))

target_para_tags = nltk.pos_tag(target_para_list)

In [557]:
def count_tags (tags_list):
    tag_fd = nltk.FreqDist(tag for (word, tag) in tags_list)
    return (tag_fd.most_common())

In [558]:
print(count_tags(target_para_tags))

[('NN', 115), ('NNP', 90), ('DT', 88), ('IN', 87), ('JJ', 44), ('VBZ', 41), ('RB', 40), ('.', 31), ('VB', 30), (',', 27), ('NNS', 24), ('TO', 23), ('CC', 20), ('POS', 16), ('VBD', 15), ('VBG', 14), ('PRP', 13), ('VBN', 12), ('PRP$', 10), ("''", 10), ('``', 9), ('VBP', 6), ('MD', 6), ('JJR', 5), ('WP', 4), ('WRB', 3), (':', 3), ('CD', 3), ('EX', 3), ('JJS', 2), ('(', 2), ('WDT', 2), (')', 2), ('RBR', 2), ('RBS', 1), ('RP', 1), ('NNPS', 1)]


In [497]:
def findtags(tag_prefix, tagged_text):
    cfd = nltk.ConditionalFreqDist((tag, word) for (word, tag) in tagged_text
                                  if tag.startswith(tag_prefix))
    return dict((tag, cfd[tag].most_common(5)) for tag in cfd.conditions())

In [560]:
findtags("NN",target_para_tags)

{'NN': [('violence', 6),
  ('lot', 4),
  ('incident', 3),
  ('explanation', 3),
  ('game', 3)],
 'NNP': [('Solo', 15),
  ('U.S.', 11),
  ('Soccer', 10),
  ('Gulati', 5),
  ('Hope', 4)],
 'NNPS': [('Americans', 1)],
 'NNS': [('women', 2),
  ('questions', 2),
  ('months', 2),
  ('concerns', 1),
  ('values', 1)]}

In [128]:
#Whether 'target Title' Start with 5W1H.
wh_start = lambda x: word_tokenize(raw_instance_train[x]['targetTitle'].lower())[0] in ['who', 'when', 'where', 'how', 'why','what']
#Ethan's edit, I delete the [0] after raw_instance_train[x]['targetTitle']. It's a string already
wh_sample = lambda x: word_tokenize('What is it'.lower())[0] in ['who', 'when', 'where', 'how', 'why','what']
print((wh_sample(0)))
print(word_tokenize(raw_instance_train[0]['targetTitle']))
print(word_tokenize(raw_instance_train[0]['targetTitle'][0]))
print(wh_start(0))

True
['Apple', 'gives', 'back', 'gigabytes', ':', 'iOS', '9', "'app", 'thinning', "'", 'feature', 'will', 'finally', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost']
['A']
False


In [109]:
#Foward reference: Whether 'target Title' has Demonstratives(this, that, those, these) 
hasDemonstrative = lambda y: sum(map(lambda x: str(x) in ['this', 'that', 'those','these'], 
                                     word_tokenize(raw_instance_train[y]['targetTitle'].lower()))) > 0
print(hasDemonstrative(0))
print(word_tokenize(raw_instance_train[0]['targetTitle'].lower()))

False
['apple', 'gives', 'back', 'gigabytes', ':', 'ios', '9', "'app", 'thinning', "'", 'feature', 'will', 'finally', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost']


In [128]:
#Foward reference: Whether 'target Title' has definite article('the')
hasDefinite = lambda y: sum(map(lambda x: str(x) == 'the', word_tokenize(raw_instance_train[y]
                                                                         ['targetTitle'].lower()))) > 0
hasDefinite(0)

True

In [117]:
#Foward reference: Whether 'target Title' has 3rd person pronoun('he','she','his','her','him')
hasThirdPronoun = lambda y: sum(map(lambda x: str(x) in ['he','she','his','her','him'], word_tokenize(raw_instance_train[y]['targetTitle'].lower()))) > 0
hasThirdPronoun(0)


False

In [156]:
#Foward reference: Whether 'target Title' Start with adverbs.
adverb_start = lambda x: pos_tag(word_tokenize(raw_instance_train[x]['targetTitle']))[0][1] == 'ADV'
# print(pos_tag(word_tokenize(raw_instance_train[0]['targetTitle']))[0][1])
print(adverb_start(0))


False


In [245]:
clickbait_train[0]['targetTitle']

'U.S. Soccer should start answering tough questions about Hope Solo'

In [246]:
clickbait_train[0]['targetParagraphs']

["WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns about the star goalkeeper.",
 "The latest lament comes from no less than a U.S. Senator, who into Solo's domestic violence incident of last year and offer a detailed explanation of why Solo is on the field. She is expected to be the starting goalkeeper when the USA plays Sweden in its second group game at the Women's World Cup on Friday.",
 '[FC Yahoo: ]',
 'U.S. Senator Richard Blumenthal of Connecticut penned a lengthy complaint about the near-silence the organization has given on Solo, especially in the wake of ESPN\'s "Outside the Lines" report on Sunday. Blumenthal wrote that if the report is accurate "U.S. Soccer\'s approach to domestic violence and family violence is at best superficial and at worst dangerously neglectful and self-serving."',
 'This situation is well beyond Solo now. U.S. Soccer has made this a referendum about its own ability to represent the values

In [312]:
def spacy_acg_sim_lg(title, body):
    body_list = []
    title = nlp_lg(title)
    for b in body:
        body_list.append(title.similarity(nlp_lg(b)))        
    return np.mean(body_list) 

def spacy_acg_sim_core_lg(title, body):
    body_list = []
    title = nlp_core_lg(title)
    for b in body:
        body_list.append(title.similarity(nlp_core_lg(b)))        
    return np.mean(body_list) 

def spacy_acg_sim(title, body):
    body_list = []
    title = nlp_en(title)
    for b in body:
        body_list.append(title.similarity(nlp_en(b)))        
    return np.mean(body_list) 

def spacy_acg_sim_en(title, body):
    body_list = []
    title = nlp(title)
    for b in body:
        body_list.append(title.similarity(nlp(b)))        
    return np.mean(body_list) 

In [313]:
print(spacy_acg_sim(clickbait_train[0]['targetTitle'], clickbait_train[0]['targetParagraphs']))
print(spacy_acg_sim_lg(clickbait_train[0]['targetTitle'], clickbait_train[0]['targetParagraphs']))
print(spacy_acg_sim_core_lg(clickbait_train[0]['targetTitle'], clickbait_train[0]['targetParagraphs']))
print(spacy_acg_sim_en(clickbait_train[0]['targetTitle'], clickbait_train[0]['targetParagraphs']))

0.713268983008
0.713268983008
0.713268983008
0.566875402212


In [277]:
print(spacy_acg_sim(clickbait_train[1]['targetTitle'], clickbait_train[1]['targetParagraphs']))
print(spacy_acg_sim_lg(clickbait_train[1]['targetTitle'], clickbait_train[1]['targetParagraphs']))
print(spacy_acg_sim_core_lg(clickbait_train[1]['targetTitle'], clickbait_train[1]['targetParagraphs']))

0.696579747105
0.696579747105
0.696579747105


In [280]:
print(spacy_acg_sim(non_clickbait_train[0]['targetTitle'], non_clickbait_train[0]['targetParagraphs']))
print(spacy_acg_sim_lg(non_clickbait_train[0]['targetTitle'], non_clickbait_train[0]['targetParagraphs']))
print(spacy_acg_sim_core_lg(non_clickbait_train[0]['targetTitle'], non_clickbait_train[0]['targetParagraphs']))

0.805061736424
0.805061736424
0.805061736424


In [318]:
bait = []
non_bait = []
for x in range(0, 100):
    bait.append(spacy_acg_sim_en(clickbait_train[x]['targetTitle'], clickbait_train[x]['targetParagraphs']))
    non_bait.append(spacy_acg_sim_en(non_clickbait_train[x]['targetTitle'], non_clickbait_train[x]['targetParagraphs']))
print(bait)
print(np.mean(bait))
print(non_bait)
print(np.mean(non_bait))

[0.56687540221182187, 0.50985086969995952, 0.47911665660326125, 0.5603878601403921, 0.59881734957342114, 0.51712832714141144, 0.37089290114414508, 0.61882848343541907, 0.23625808029908985, 0.59168612024112233, 0.36493104801633119, 0.50633185574348405, 0.57726160156698081, 0.14982355632069932, 0.47559553618131384, 0.47980012819828349, 0.50439637931267156, 0.31338355078344143, 0.67130023539476247, 0.65597724463824225, 0.58835304069642147, 0.74504834685644739, 0.58044116675746593, 0.43465435359670701, 0.42675142208514855, 0.64152274653532293, 0.48751852812505048, 0.53094540055102335, 0.63605935285367388, 0.72131931845542552, 0.54126572071383949, 0.58710082084990378, 0.59526088798802401, 0.45283390276903968, 0.38646000554808257, 0.27874326885828604, 0.59489120874645107, 0.46349992281546937, 0.49155166787859311, 0.6550014953349137, 0.64515822731164896, 0.140578053900936, 0.5281877175407933, 0.64100202964790143, 0.49362951167069574, 0.65735084106997144, 0.67395831716178023, 0.710916285829355

In [310]:
import spacy
nlp = spacy.load('en')
 
doc1 = nlp(u'Hello this is document similarity calculation')
doc2 = nlp(u'Hello this is python similarity calculation')
doc3 = nlp(u'Hi there')
 
print (doc1.similarity(doc2)) 
print (doc2.similarity(doc3)) 
print (doc1.similarity(doc3))  

0.940947083805
0.326296786973
0.295776641004


In [317]:
import spacy
nlp = spacy.load('en')
 
doc1 = nlp('Hello this is document similarity calculation')
doc2 = nlp('Hello this is python similarity calculation')
doc3 = nlp('Hi there')
 
print (doc1.similarity(doc2)) 
print (doc2.similarity(doc3)) 
print (doc1.similarity(doc3))  

0.940947083805
0.326296786973
0.295776641004


In [316]:
nlp = spacy.load('en', vectors='en_glove_cc_300_1m')

In [322]:
from gensim.models import Word2Vec
# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
dog = model['dog']
print(dog.shape)
print(dog[:10])

# Deal with an out of dictionary word: Михаил (Michail)
if 'Михаил' in model:
    print(model['Михаил'].shape)
else:
    print('{0} is an out of dictionary word'.format('Михаил'))

# Some predefined functions that show content related information for given words
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

print(model.doesnt_match("breakfast cereal dinner lunch".split()))

print(model.similarity('woman', 'man'))

(300,)
[ 0.05126953 -0.02233887 -0.17285156  0.16113281 -0.08447266  0.05737305
  0.05859375 -0.08251953 -0.01538086 -0.06347656]
Михаил is an out of dictionary word
[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431607246399), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.5181134343147278), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454)]
cereal
0.766401


In [329]:
import gensim
print(dir(gensim))

['NullHandler', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_matutils', 'corpora', 'downloader', 'interfaces', 'logger', 'logging', 'matutils', 'models', 'parsing', 'scripts', 'similarities', 'summarization', 'test', 'topic_coherence', 'utils']


In [330]:
def gensim_sim(title, body)


raw_documents = ["I'm taking the show on the road.",
                 "My socks are a force multiplier.",
             "I am the barber who cuts everyone's hair who doesn't cut their own.",
             "Legend has it that the mind is a mad monkey.",
            "I make my own fun."]
print("Number of documents:",len(raw_documents))

Number of documents: 5


In [331]:
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]
print(gen_docs)

[['i', "'m", 'taking', 'the', 'show', 'on', 'the', 'road', '.'], ['my', 'socks', 'are', 'a', 'force', 'multiplier', '.'], ['i', 'am', 'the', 'barber', 'who', 'cuts', 'everyone', "'s", 'hair', 'who', 'does', "n't", 'cut', 'their', 'own', '.'], ['legend', 'has', 'it', 'that', 'the', 'mind', 'is', 'a', 'mad', 'monkey', '.'], ['i', 'make', 'my', 'own', 'fun', '.']]


In [398]:
def gensim_sim(title, body):
    gen_docs = [[w.lower() for w in word_tokenize(text)] for text in body]
    dictionary = gensim.corpora.Dictionary(gen_docs)
    corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
    print(dictionary)
    tf_idf = gensim.models.TfidfModel(corpus)
    sims = gensim.similarities.Similarity(None,tf_idf[corpus],num_features=len(dictionary))
    query_doc = [w.lower() for w in word_tokenize(title)]
    query_doc_bow = dictionary.doc2bow(query_doc)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    return np.mean(sims[query_doc_tf_idf])

In [408]:
aaa = []
for x in range (2000):
    sim_val = gensim_sim(clickbait_train[x]['targetTitle'],clickbait_train[x]['targetParagraphs'])
    if sim_val > 0:
        aaa.append(sim_val)
np.mean(aaa)

Dictionary(341 unique tokens: ["'s", ',', '.', 'about', 'anyone']...)
Dictionary(420 unique tokens: ["'s", '(', ')', '.', 'always']...)
Dictionary(311 unique tokens: ['.', '1', 'another', 'artist', 'is']...)
Dictionary(289 unique tokens: ['.', 'a', 'are', 'for', 'getting']...)
Dictionary(569 unique tokens: ["''", ',', '.', '``', 'a']...)
Dictionary(165 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(65 unique tokens: [',', '.', 'a', 'and', 'barrier']...)
Dictionary(64 unique tokens: ["''", "'s", ',', '.', '``']...)
Dictionary(281 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(706 unique tokens: [',', '.', 'a', 'and', 'are']...)
Dictionary(49 unique tokens: ["''", ',', '--', '.', '``']...)
Dictionary(547 unique tokens: ['(', ')', ',', ':', 'beach']...)
Dictionary(120 unique tokens: [',', '.', '2014', 'adherents', 'aiding']...)
Dictionary(8 unique tokens: ['.', 'are', 'available', 'currently', 'no']...)
Dictionary(437 unique toke

Dictionary(214 unique tokens: [',', '.', 'and', 'at', 'car']...)
Dictionary(291 unique tokens: ["'", '(', ')', ',', '-']...)
Dictionary(175 unique tokens: [',', '.', 'against', 'before', 'bernie']...)
Dictionary(399 unique tokens: [',', '.', '2016', 'a', 'american']...)
Dictionary(132 unique tokens: [',', '.', 'a', 'according', 'climbing']...)
Dictionary(157 unique tokens: [',', 'and', 'be', 'but', 'come']...)
Dictionary(282 unique tokens: [',', 'a', 'african', 'american', 'as']...)
Dictionary(236 unique tokens: ["'s", ',', '.', '2015', 'a']...)
Dictionary(121 unique tokens: ['.', 'a', 'after', 'an', 'and']...)
Dictionary(121 unique tokens: [',', '.', '17', 'at', 'cheer']...)
Dictionary(251 unique tokens: ["''", "'s", ',', '.', '``']...)
Dictionary(147 unique tokens: [',', '.', 'a', 'and', 'angry']...)
Dictionary(338 unique tokens: ['amid', 'approves', 'by', 'computerised', 'drivers']...)
Dictionary(437 unique tokens: ['!', ',', '.', 'an', 'as']...)
Dictionary(239 unique tokens: ["'s",

Dictionary(184 unique tokens: ['by', 'foley', 'kaye', ',', '.']...)
Dictionary(69 unique tokens: [',', '.', '?', 'a', 'and']...)
Dictionary(422 unique tokens: ['!', '(', ')', ',', '.']...)
Dictionary(749 unique tokens: [',', '.', '14', '73', 'and']...)
Dictionary(215 unique tokens: ["'s", ',', '--', '.', ':']...)
Dictionary(55 unique tokens: [',', '.', 'accident', 'an', 'between']...)
Dictionary(149 unique tokens: ['.', '20.29sec', '200m', 'a', 'adidas']...)
Dictionary(269 unique tokens: ['.', '13', 'a', 'belgium', 'doctors']...)
Dictionary(114 unique tokens: ["''", ',', '.', ';', '``']...)
Dictionary(171 unique tokens: ["'s", '(', ')', ',', '-']...)
Dictionary(302 unique tokens: ["'s", ',', '.', 'a', 'airstrikes']...)
Dictionary(241 unique tokens: ['#', ',', '.', 'and', 'archeologists']...)
Dictionary(226 unique tokens: ["'s", '.', 'a', 'and', 'been']...)
Dictionary(174 unique tokens: ['.', 'a', 'ago', 'as', 'declared']...)
Dictionary(623 unique tokens: [',', '.', '?', 'after', 'and']

Dictionary(146 unique tokens: ['#', ',', '--', '.', 'a']...)
Dictionary(529 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(376 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(124 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(97 unique tokens: [',', '.', 'a', 'and', 'athens']...)
Dictionary(310 unique tokens: ["'s", ',', '.', '86', 'age']...)
Dictionary(233 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(195 unique tokens: ['.', 'candidate', 'clinton', 'comes', 'democratic']...)
Dictionary(157 unique tokens: ['circus', 'elephant', 'escaped', 'from', 'had']...)
Dictionary(119 unique tokens: ['(', ')', ',', '.', 'and']...)
Dictionary(105 unique tokens: ['--', '.', 'a', 'and', 'grab']...)
Dictionary(412 unique tokens: [',', '.', '80', 'a', 'angeles']...)
Dictionary(120 unique tokens: ["'", ',', '.', 'a', 'abuse']...)
Dictionary(97 unique tokens: [',', '.', 'a', 'accepted',

Dictionary(246 unique tokens: [',', '.', 'a', 'agenda', 'and']...)
Dictionary(149 unique tokens: ['.', 'a', 'avalanche', 'emerged', 'escaping']...)
Dictionary(209 unique tokens: ['.', 'a', 'after', 'broke', 'dave']...)
Dictionary(148 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(166 unique tokens: [',', '.', '2007', 'an', 'and']...)
Dictionary(77 unique tokens: ["'s", ',', '.', 'a', 'an']...)
Dictionary(708 unique tokens: ["''", ',', '.', '27', '``']...)
Dictionary(181 unique tokens: ["'re", ',', '--', '.', '3-d']...)
Dictionary(245 unique tokens: ['.', 'as', 'chief', 'dorsey', 'executive']...)
Dictionary(57 unique tokens: ['.', '11th-hour', 'a', 'agenda', 'ahead']...)
Dictionary(317 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(1072 unique tokens: ['.', 'a', 'azalea', 'done', 'fork']...)
Dictionary(124 unique tokens: ['.', 'a', 'captured', 'close-up', 'footage']...)
Dictionary(339 unique tokens: ['.', ';', 'an', 'ancient', 

...

Dictionary(1483 unique tokens: [',', '.', ':', 'alike', 'all-star']...)
Dictionary(50 unique tokens: ['.', 'and', 'are', 'bowl', 'championships']...)
Dictionary(496 unique tokens: [',', '.', 'are', 'around', 'first-time']...)
Dictionary(222 unique tokens: ['and', 'brexit', 'can', 'case', 'decides']...)
Dictionary(224 unique tokens: [',', '.', '7', 'a', 'all']...)
Dictionary(521 unique tokens: ['more', ',', '.', 'a', 'airlines']...)
Dictionary(451 unique tokens: [',', '.', 'a', 'address', 'america']...)
Dictionary(83 unique tokens: ['danielle', 'muoio', ',', '15:37', '21.01.2017']...)
Dictionary(608 unique tokens: ['creditdoug', 'mills/the', 'new', 'times', 'york']...)
Dictionary(544 unique tokens: [',', '.', '40th', 'a', 'and']...)
Dictionary(246 unique tokens: ["'", "'we", 'all', 'and', 'hands']...)
Dictionary(181 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(116 unique tokens: [',', '.', 'and', 'are', 'bloomberg']...)
Dictionary(256 unique tokens: [':',

Dictionary(411 unique tokens: ["'s", ',', '.', 'a', 'admired']...)
Dictionary(480 unique tokens: ['$', ',', '.', '100-million', '?']...)
Dictionary(674 unique tokens: ['.', 'and', 'as', 'austrian', 'candidate']...)
Dictionary(93 unique tokens: ['gillett', 'lee', 'rachel', 'samantha', 'und']...)
Dictionary(443 unique tokens: ["'s", '(', ')', ',', '--']...)
Dictionary(216 unique tokens: [',', '.', '60', '80', 'a']...)
Dictionary(144 unique tokens: ['#', ',', '.', '2016', 'a']...)
Dictionary(87 unique tokens: ['.', 'bulls', 'conchi', 'female', 'fighting']...)
Dictionary(811 unique tokens: ['vancouver', 'canada', 'cascadia', 'seattle', 'pacific']...)
Dictionary(458 unique tokens: ['--', '.', 'a', 'alleged', 'and']...)
Dictionary(304 unique tokens: ['$', "'", "'s", '--', '.']...)
Dictionary(400 unique tokens: ['1', 'related', ',', '.', 'a']...)
Dictionary(226 unique tokens: [',', '.', 'a', 'after', 'alabama']...)
Dictionary(281 unique tokens: [',', '.', 'a', 'an', 'annual']...)
Dictionary(3

Dictionary(234 unique tokens: ["''", "'ve", ',', '.', '...']...)
Dictionary(176 unique tokens: [',', '.', 'a', 'acosta', 'advertising']...)
Dictionary(553 unique tokens: ["'", "'underwater", '30', 'already', 'earth']...)
Dictionary(165 unique tokens: ["''", "'s", ',', '.', '``']...)
Dictionary(180 unique tokens: ['.', 'after', 'cbs', 'charlie', 'co-host']...)
Dictionary(177 unique tokens: ['#', '?', '@', 'als', 'andylenz']...)
Dictionary(174 unique tokens: [',', 'business', 'insider', 'jacobs', 'sarah']...)
Dictionary(176 unique tokens: [',', '.', 'a', 'according', 'and']...)
Dictionary(381 unique tokens: ['more', '#', "'re", "'s", '.']...)
Dictionary(59 unique tokens: [',', '.', 'a', 'allegedly', 'alphabet']...)
Dictionary(140 unique tokens: ["''", ',', '.', '2-year-old', '``']...)
Dictionary(109 unique tokens: [',', '.', 'administration', 'agenda', 'american']...)
Dictionary(206 unique tokens: [',', '.', '2005', 'adviser', 'and']...)
Dictionary(271 unique tokens: [',', '.', '19', 'a'

Dictionary(175 unique tokens: [',', 'business', 'guerrasio', 'insider', 'jason']...)
Dictionary(155 unique tokens: [',', '.', '9', 'a.m.', 'about']...)
Dictionary(173 unique tokens: ["'", "''", "'born", "'purposeful", "'re"]...)
Dictionary(210 unique tokens: [',', '.', 'a', 'against', 'anthony']...)
Dictionary(114 unique tokens: ['.', 'on', 'or', 'says', 'she']...)
Dictionary(264 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(126 unique tokens: [',', '2017', '31', '6:59', 'est']...)
Dictionary(125 unique tokens: ['.', 'administration', 'back', 'has', 'house']...)
Dictionary(194 unique tokens: ["'s", '.', 'a', 'advising', 'amending']...)
Dictionary(440 unique tokens: ["'s", ',', '.', 'a', 'accompanied']...)
Dictionary(169 unique tokens: [',', '.', 'alphabet', 'and', 'are']...)
Dictionary(157 unique tokens: ["'s", ',', '.', 'a', 'all']...)
Dictionary(57 unique tokens: [',', '.', 'and', 'are', 'at']...)
Dictionary(437 unique tokens: [':', 'attorney', 'debate'

0.076320656

In [409]:
bbb = []
for x in range (2000):
    sim_val = gensim_sim(non_clickbait_train[x]['targetTitle'],non_clickbait_train[x]['targetParagraphs'])
    if sim_val > 0:
        bbb.append(sim_val)
np.mean(bbb)

Dictionary(328 unique tokens: ["'s", '.', '64gb', 'a', 'and']...)
Dictionary(51 unique tokens: ['.', 'are', 'emerging', 'favor', 'markets']...)
Dictionary(765 unique tokens: ['$', ',', '.', '1971', '3.50']...)
Dictionary(460 unique tokens: [',', '.', 'a', 'after', 'agency']...)
Dictionary(89 unique tokens: [',', '.', '12', 'a', 'after']...)
Dictionary(49 unique tokens: [',', '.', '10', 'a', 'abroad']...)
Dictionary(358 unique tokens: ["'s", ',', '.', 'a', 'agenda']...)
Dictionary(204 unique tokens: [',', '.', 'a', 'and', 'attending']...)
Dictionary(186 unique tokens: [',', '90', 'a', 'actually', 'believe']...)
Dictionary(448 unique tokens: [',', 'dear', 'jerry', 'seinfeld', "''"]...)
Dictionary(357 unique tokens: ["'s", 'actions', 'activity', 'anti-terror', 'chiefs']...)
Dictionary(372 unique tokens: ['$', ',', '.', '100', 'a']...)
Dictionary(384 unique tokens: ['.', 'against', 'alberto', 'allegations', 'american']...)
Dictionary(168 unique tokens: [',', '.', 'a', 'and', 'armpits']...)

Dictionary(300 unique tokens: ["'re", '.', ':', 'are', 'following']...)
Dictionary(382 unique tokens: ['(', ')', ',', '/', '8-9']...)
Dictionary(495 unique tokens: ['.', 'being', 'empire', 'hand', 'his']...)
Dictionary(60 unique tokens: ['.', 'all', 'at', 'found', 'local']...)
Dictionary(99 unique tokens: ['(', ')', '.', '2015', '?']...)
Dictionary(734 unique tokens: ['.', 'a', 'activity', 'allegedly', 'as']...)
Dictionary(274 unique tokens: ["''", "'s", ',', '.', '1']...)
Dictionary(661 unique tokens: ['.', 'a', 'as', 'bars', 'blair']...)
Dictionary(210 unique tokens: ["'s", '.', 'and', 'artist', 'discovered']...)
Dictionary(165 unique tokens: ["'s", ',', '.', 'for', 'friday']...)
Dictionary(743 unique tokens: [',', '.', 'a', 'and', 'bare']...)
Dictionary(83 unique tokens: [',', '--', '.', 'a', 'again']...)
Dictionary(278 unique tokens: ["'s", '?', 'be', 'ceo', 'influential']...)
Dictionary(324 unique tokens: ['(', ')', '.', ':', 'a']...)
Dictionary(428 unique tokens: ['.', 'ebola', '

Dictionary(512 unique tokens: [',', '.', 'a', 'administration', 'against']...)
Dictionary(115 unique tokens: ['(', ')', ',', '.', 'a']...)
Dictionary(267 unique tokens: [',', '.', 'actor', 'as', 'barry']...)
Dictionary(358 unique tokens: ["'s", ',', '.', 'a', 'agenda']...)
Dictionary(137 unique tokens: ['.', 'against', 'basketball', 'guy', 'had']...)
Dictionary(251 unique tokens: ["'s", '.', 'an', 'armored', 'dallas']...)
Dictionary(575 unique tokens: [',', '.', '2000s', '73', 'a']...)
Dictionary(208 unique tokens: [',', '.', '14,609', '21', '72']...)
Dictionary(149 unique tokens: ['.', '69', 'died', 'dusty', 'he']...)
Dictionary(188 unique tokens: ['.', 'a', 'been', 'britain', 'campaign']...)
Dictionary(180 unique tokens: ["'", "'nail", '.', 'and', 'be']...)
Dictionary(96 unique tokens: ["'s", ',', 'a', 'bites', 'boulder']...)
Dictionary(478 unique tokens: [',', '.', '7-year-old', '90s', 'a']...)
Dictionary(432 unique tokens: ["'s", ',', '--', '.', '34']...)
Dictionary(124 unique toke

Dictionary(685 unique tokens: [',', '.', 'a', 'actor', 'and']...)
Dictionary(326 unique tokens: [',', '.', 'a', 'baby', 'carrying']...)
Dictionary(166 unique tokens: [',', '.', 'a', 'almost', 'aspect']...)
Dictionary(389 unique tokens: ["'s", ',', '.', 'a', 'always']...)
Dictionary(331 unique tokens: [',', '.', 'after', 'an', 'announcing']...)
Dictionary(250 unique tokens: ['after', 'airport', 'at', 'charles', 'child']...)
Dictionary(246 unique tokens: ['.', 'at', 'comes', 'fast', 'life']...)
Dictionary(415 unique tokens: ["''", '.', ':', '``', 'a']...)
Dictionary(75 unique tokens: ['.', 'a', 'actor', 'and', 'baby']...)
Dictionary(452 unique tokens: ['%', "'s", "'ve", ',', '.']...)
Dictionary(99 unique tokens: ['(', ')', '@', 'a', 'by']...)
Dictionary(178 unique tokens: ['.', 'and', 'barbie', 'be', 'follow']...)
Dictionary(290 unique tokens: ['.', '75', 'and', 'appear', 'at']...)
Dictionary(215 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(75 unique token

Dictionary(463 unique tokens: ["''", '-', '?', '``', 'asylum']...)
Dictionary(122 unique tokens: ["'", '.', 'best', 'collins', 'former']...)
Dictionary(259 unique tokens: ["''", '.', '``', 'a', 'abused']...)
Dictionary(219 unique tokens: [',', '2012', 'and', 'as', 'ayotte']...)
Dictionary(289 unique tokens: [',', '.', '2-year-old', 'a', 'and']...)
Dictionary(202 unique tokens: ["''", "'re", "'s", ',', '.']...)
Dictionary(1229 unique tokens: ['.', '2', 'a', 'be', 'but']...)
Dictionary(356 unique tokens: ["''", '.', '``', 'a', 'around']...)
Dictionary(180 unique tokens: ["''", ',', '.', '81', '``']...)
Dictionary(76 unique tokens: [',', '.', '13-year-old', 'a', 'and']...)
Dictionary(96 unique tokens: [',', '.', 'announced', 'as', 'ceo']...)
Dictionary(634 unique tokens: ['(', ')', ',', '.', '1970s']...)
Dictionary(448 unique tokens: [',', '.', '24', '37,000', 'a']...)
Dictionary(146 unique tokens: [',', '.', 'a', 'and', 'backyards']...)
Dictionary(54 unique tokens: ['$', ',', '.', '580-a

...

Dictionary(499 unique tokens: ['.', 'a', 'and', 'are', 'backing']...)
Dictionary(564 unique tokens: [',', '.', 'a', 'against', 'allegations']...)
Dictionary(324 unique tokens: [',', '.', 'any', 'away', 'been']...)
Dictionary(819 unique tokens: ['accidentally', 'donated', 'dress', 'for', 'husband']...)
Dictionary(118 unique tokens: [',', 'business', 'insider', 'jethro', 'nededog']...)
Dictionary(144 unique tokens: ['(', ')', ',', '.', 'a']...)
Dictionary(249 unique tokens: ['.', 'a', 'and', 'are', 'be']...)
Dictionary(1191 unique tokens: ["''", "'s", ',', '.', '?']...)
Dictionary(73 unique tokens: [',', '.', 'enough', 'for', 'funds']...)
Dictionary(237 unique tokens: ['#', "'s", '(', ')', ',']...)
Dictionary(155 unique tokens: ["'s", '.', 'a', 'back', 'been']...)
Dictionary(431 unique tokens: [',', '.', ';', 'a', 'agrawal']...)
Dictionary(412 unique tokens: [',', 'about', 'bartending', 'breaking', 'cleaning']...)
Dictionary(138 unique tokens: ["''", "'s", '(', ')', ',']...)
Dictionary(9

Dictionary(1191 unique tokens: ['.', 'an', 'dealing', 'epic', 'friday']...)
Dictionary(301 unique tokens: ['-', '.', 'a', 'app', 'brief']...)
Dictionary(125 unique tokens: ['(', ')', '.', 'at', 'colbert']...)
Dictionary(212 unique tokens: ['.', ':', 'blatantly', 'clear', 'dan']...)
Dictionary(284 unique tokens: [',', '.', 'a', 'actually', 'and']...)
Dictionary(238 unique tokens: [',', 'alone', 'be', 'better', 'bread']...)
Dictionary(225 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(86 unique tokens: [',', 'business', 'elena', 'holodny', 'insider']...)
Dictionary(338 unique tokens: ["'s", ',', '.', '8,000', 'access']...)
Dictionary(522 unique tokens: ['.', 'after', 'be', 'become', 'country']...)
Dictionary(374 unique tokens: [',', '.', 'a', 'achy-breaky', 'acquainted']...)
Dictionary(121 unique tokens: ["''", "'s", '(', ')', ',']...)
Dictionary(632 unique tokens: [',', '.', '1996', 'a', 'accept']...)
Dictionary(123 unique tokens: ["''", ',', '.', '``', 'al

Dictionary(235 unique tokens: [',', ':', 'and', 'at', 'bannon']...)
Dictionary(286 unique tokens: [',', '.', 'a', 'accept', 'administration']...)
Dictionary(115 unique tokens: ['.', 'a', 'an', 'attack', 'brain']...)
Dictionary(123 unique tokens: ['(', ')', '.', 'a', 'and']...)
Dictionary(224 unique tokens: ['(', ')', ',', '.', 'a']...)
Dictionary(448 unique tokens: [',', '.', 'about', 'according', 'among']...)
Dictionary(164 unique tokens: [',', '--', '.', 'a', 'aaron']...)
Dictionary(665 unique tokens: ['.', 'a', 'acoustic', 'album', 'christo']...)
Dictionary(1061 unique tokens: ["'re", "'s", '(', ')', '.']...)
Dictionary(200 unique tokens: [',', 'abby', 'business', 'insider', 'jackson']...)
Dictionary(584 unique tokens: ['shutterstock', "'s", '(', ')', ',']...)
Dictionary(97 unique tokens: ["'s", ',', '.', '3000', ':']...)
Dictionary(207 unique tokens: ['.', 'a', 'after', 'allegedly', 'attacked']...)
Dictionary(486 unique tokens: ['more', '(', ')', ',', '.']...)
Dictionary(260 unique

Dictionary(289 unique tokens: [',', '.', 'about', 'app', 'flipboard']...)
Dictionary(404 unique tokens: [',', '.', 'a', 'ark.', 'at']...)
Dictionary(252 unique tokens: ["''", "'s", '.', '1', ':']...)
Dictionary(351 unique tokens: ['.', 'a', 'accused', 'attorney', 'condemned']...)
Dictionary(63 unique tokens: ["'s", ',', '.', 'activist', 'bbc']...)
Dictionary(54 unique tokens: ['$', ',', '.', '1.8', 'a']...)
Dictionary(327 unique tokens: [',', '.', 'a', 'about', 'after']...)
Dictionary(295 unique tokens: ['share', 'this', 'with', 'email', 'facebook']...)
Dictionary(342 unique tokens: [',', '.', 'a', 'about', 'book']...)
Dictionary(91 unique tokens: ['.', 'about', 'ending', 'health', 'is']...)
Dictionary(356 unique tokens: [',', 'advances', 'an', 'books', 'children']...)
Dictionary(212 unique tokens: [',', '.', 'a', 'allow', 'american']...)
Dictionary(42 unique tokens: [',', 'big', 'journalism', 'milo', "'s"]...)
Dictionary(422 unique tokens: [',', '.', 'a', 'according', 'after']...)
Dic

0.078901097

In [332]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary[5])
print(dictionary.token2id['road'])
print("Number of words in dictionary:",len(dictionary))
for i in range(len(dictionary)):
    print(i, dictionary[i])

show
4
Number of words in dictionary: 36
0 'm
1 .
2 i
3 on
4 road
5 show
6 taking
7 the
8 a
9 are
10 force
11 multiplier
12 my
13 socks
14 's
15 am
16 barber
17 cut
18 cuts
19 does
20 everyone
21 hair
22 n't
23 own
24 their
25 who
26 has
27 is
28 it
29 legend
30 mad
31 mind
32 monkey
33 that
34 fun
35 make


In [333]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2)], [(1, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)], [(1, 1), (2, 1), (7, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2)], [(1, 1), (7, 1), (8, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)], [(1, 1), (2, 1), (12, 1), (23, 1), (34, 1), (35, 1)]]


In [334]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=5, num_nnz=47)
47


In [346]:
sims = gensim.similarities.Similarity(None,tf_idf[corpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

Similarity index with 5 documents in 0 shards (stored under /var/folders/0f/qdj5sql91zd4g4xf9scp_jx80000gq/T/simserver1a8bc5)
<class 'gensim.similarities.docsim.Similarity'>


In [347]:
query_doc = [w.lower() for w in word_tokenize("Socks are a force for good.")]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

['socks', 'are', 'a', 'force', 'for', 'good', '.']
[(1, 1), (8, 1), (9, 1), (10, 1), (13, 1)]
[(8, 0.31226270667960454), (9, 0.54848032538919966), (10, 0.54848032538919966), (13, 0.54848032538919966)]


In [348]:
sims[query_doc_tf_idf]

array([ 0.        ,  0.84565616,  0.        ,  0.06124881,  0.        ], dtype=float32)

In [ ]:
clickbait_train[0]['targetTitle']
clickbait_train[0]['targetParagraphs']

In [457]:
clickbait_train[0]['targetParagraphs'][0]

"WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns about the star goalkeeper."

In [456]:
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in clickbait_train[0]['targetParagraphs'] ]
print((gen_docs[0]))
gen_docs_ = [[a for a in c if a not in punctuation] for c in gen_docs ]
print(gen_docs_)

['winnipeg', ',', 'manitoba', '–', 'the', 'bubble', 'u.s.', 'soccer', 'is', 'putting', 'around', 'hope', 'solo', 'is', "n't", 'working', 'to', 'calm', 'anyone', "'s", 'concerns', 'about', 'the', 'star', 'goalkeeper', '.']
[['winnipeg', 'manitoba', '–', 'the', 'bubble', 'u.s.', 'soccer', 'is', 'putting', 'around', 'hope', 'solo', 'is', "n't", 'working', 'to', 'calm', 'anyone', "'s", 'concerns', 'about', 'the', 'star', 'goalkeeper'], ['the', 'latest', 'lament', 'comes', 'from', 'no', 'less', 'than', 'a', 'u.s', 'senator', 'who', 'into', 'solo', "'s", 'domestic', 'violence', 'incident', 'of', 'last', 'year', 'and', 'offer', 'a', 'detailed', 'explanation', 'of', 'why', 'solo', 'is', 'on', 'the', 'field', 'she', 'is', 'expected', 'to', 'be', 'the', 'starting', 'goalkeeper', 'when', 'the', 'usa', 'plays', 'sweden', 'in', 'its', 'second', 'group', 'game', 'at', 'the', 'women', "'s", 'world', 'cup', 'on', 'friday'], ['fc', 'yahoo'], ['u.s', 'senator', 'richard', 'blumenthal', 'of', 'connecticu

In [440]:
dictionary_ = gensim.corpora.Dictionary(gen_docs_)
print(dictionary_[5])
print(dictionary_.token2id['about'])
print("Number of words in dictionary:",len(dictionary_))
for i in range(len(dictionary_)):
    print(i, dictionary_[i])

around
3
Number of words in dictionary: 341
0 's
1 ,
2 .
3 about
4 anyone
5 around
6 bubble
7 calm
8 concerns
9 goalkeeper
10 hope
11 is
12 manitoba
13 n't
14 putting
15 soccer
16 solo
17 star
18 the
19 to
20 u.s.
21 winnipeg
22 working
23 –
24 a
25 and
26 at
27 be
28 comes
29 cup
30 detailed
31 domestic
32 expected
33 explanation
34 field
35 friday
36 from
37 game
38 group
39 in
40 incident
41 into
42 its
43 lament
44 last
45 latest
46 less
47 no
48 of
49 offer
50 on
51 plays
52 second
53 senator
54 she
55 starting
56 sweden
57 than
58 u.s
59 usa
60 violence
61 when
62 who
63 why
64 women
65 world
66 year
67 :
68 [
69 ]
70 fc
71 yahoo
72 ''
73 ``
74 accurate
75 approach
76 best
77 blumenthal
78 complaint
79 connecticut
80 dangerously
81 especially
82 espn
83 family
84 given
85 has
86 if
87 lengthy
88 lines
89 near-silence
90 neglectful
91 organization
92 outside
93 penned
94 report
95 richard
96 self-serving
97 sunday
98 superficial
99 that
100 wake
101 worst
102 wrote
103 ?
104 abili

In [352]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)], [(0, 2), (1, 1), (2, 3), (9, 1), (11, 2), (16, 2), (18, 5), (19, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1)], [(67, 1), (68, 1), (69, 1), (70, 1), (71, 1)], [(0, 2), (1, 1), (2, 3), (3, 1), (11, 2), (15, 1), (16, 1), (18, 5), (19, 1), (20, 1), (24, 1), (25, 3), (26, 2), (31, 1), (39, 1), (48, 2), (50, 2), (53, 1), (58, 1), (60, 2), (72, 2), (73, 2), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (8

In [353]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=16, num_nnz=625)
625


In [354]:
sims = gensim.similarities.Similarity(None,tf_idf[corpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

Similarity index with 16 documents in 0 shards (stored under /var/folders/0f/qdj5sql91zd4g4xf9scp_jx80000gq/T/simservercc1217)
<class 'gensim.similarities.docsim.Similarity'>


In [356]:
query_doc = [w.lower() for w in word_tokenize(clickbait_train[0]['targetTitle'])]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

['u.s.', 'soccer', 'should', 'start', 'answering', 'tough', 'questions', 'about', 'hope', 'solo']
[(3, 1), (10, 1), (15, 1), (16, 1), (20, 1), (151, 1), (322, 1)]
[(3, 0.4481199656584941), (10, 0.37110808075842094), (15, 0.12581898166127858), (16, 0.1003046472630994), (20, 0.12581898166127858), (151, 0.55666212113763147), (322, 0.55666212113763147)]


In [357]:
sims[query_doc_tf_idf]

array([ 0.14804909,  0.00514506,  0.        ,  0.05579161,  0.09988002,
        0.        ,  0.08448693,  0.00369259,  0.04070854,  0.06461674,
        0.00388119,  0.04364515,  0.01308389,  0.07119576,  0.08902098,  0.        ], dtype=float32)

In [358]:
np.mean(sims[query_doc_tf_idf])

0.045199849